In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sea
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import xgboost as xgb
import tensorflow as tf


In [2]:
df = pd.read_csv('SolAtasIMC_tratado.csv')
df.head()

,date,open,high,low,close,value
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,75
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,75
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,75
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,75
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,75


In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36400 entries, 0 to 36399
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    36400 non-null  object 
 1   open    36400 non-null  float64
 2   high    36400 non-null  float64
 3   low     36400 non-null  float64
 4   close   36400 non-null  float64
 5   value   36400 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 1.7+ MB
None


# Preprocesado de Datos

In [4]:
tamanio = df.shape[0]

In [5]:
df_train = df.copy().loc[0:int(tamanio*0.7)]
df_train

,date,open,high,low,close,value
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,75
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,75
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,75
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,75
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,75
...,...,...,...,...,...,...
25476,2023-07-11 21:00:00,22.00,22.05,21.90,22.00,57
25477,2023-07-11 22:00:00,22.00,22.10,21.90,22.00,57
25478,2023-07-11 23:00:00,22.00,22.05,21.75,21.95,57
25479,2023-07-12 00:00:00,21.95,22.10,21.90,22.05,64


In [6]:
df_vali = df.copy().loc[int(tamanio*0.7 + 1):int(tamanio*0.9)]
df_vali

,date,open,high,low,close,value
25481,2023-07-12 02:00:00,22.10,22.30,22.05,22.15,64
25482,2023-07-12 03:00:00,22.15,22.25,22.10,22.10,64
25483,2023-07-12 04:00:00,22.10,22.10,22.00,22.00,64
25484,2023-07-12 05:00:00,22.00,22.00,21.90,21.95,64
25485,2023-07-12 06:00:00,21.95,22.05,21.90,22.00,64
...,...,...,...,...,...,...
32756,2024-05-10 05:00:00,153.65,154.35,152.85,153.95,66
32757,2024-05-10 06:00:00,153.95,154.70,153.45,153.75,66
32758,2024-05-10 07:00:00,153.75,154.10,152.30,153.30,66
32759,2024-05-10 08:00:00,153.30,155.10,153.15,154.95,66


In [7]:
df_test = df.copy().loc[int(tamanio*0.9 + 1):tamanio]
df_test

,date,open,high,low,close,value
32761,2024-05-10 10:00:00,154.35,154.5,153.45,154.10,66
32762,2024-05-10 11:00:00,154.10,154.8,153.25,154.15,66
32763,2024-05-10 12:00:00,154.15,154.3,153.25,154.15,66
32764,2024-05-10 13:00:00,154.15,155.2,153.00,155.05,66
32765,2024-05-10 14:00:00,155.05,155.4,153.10,153.30,66
...,...,...,...,...,...,...
36395,2024-10-08 20:00:00,143.35,143.9,142.35,142.95,49
36396,2024-10-08 21:00:00,142.95,144.1,142.25,143.75,49
36397,2024-10-08 22:00:00,143.75,144.5,143.35,144.50,49
36398,2024-10-08 23:00:00,144.50,144.7,144.05,144.25,49


In [8]:
df_valitest = pd.concat([df_vali, df_test], axis=0)

Numero de horas que se utilizan en la predicción

In [9]:
numhorasconst = 4

# Declaramos y Entrenamos el modelo

Función que prepara los datos

In [10]:
def preparar_datosRandomForest(df, numhoras):
    X = []
    y= []
    for i in range(0, df.shape[0] - numhoras):
        auxy = df.iloc[i + numhoras]
        y.append(auxy.close)
        aux1 = []
        for e in range(0, numhoras):
            aux = df.iloc[i + e]
            for r in range(1, aux.size):
                aux1.append(aux[r])
        X.append(aux1)
    return (X, y)

Preparamos los datos en el formato requerido por el algoritmo para su entrenamiento

In [11]:
data = preparar_datosRandomForest(df_train, numhorasconst)
X1 = data[0]
y1 = data[1]

C:\Users\raulg\AppData\Local\Temp\ipykernel_5376\2914413355.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux1.append(aux[r])


Preparamos los datos usados para testear el algortimo en el formato requerido

In [12]:
data = preparar_datosRandomForest(df_valitest, numhorasconst)
Xvt = data[0]
yvt = data[1]

C:\Users\raulg\AppData\Local\Temp\ipykernel_5376\2914413355.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux1.append(aux[r])


Desarrollamos un algoritmo para entrenar distintas instacias con distintos parámetros y compararlas

In [13]:
def evalRandomForest(Testrpr, predictT):
    suma = 0
    n = len(Testrpr)
    for i in range(0,n):
        suma = abs(predictT[i] - Testrpr[i])/Testrpr[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

In [14]:
def train_randomForestdepth(ini, fin, Xtr, ytr, Xvtaux, yvtaux):
    posbest = 0
    best = 100
    for i in range(ini, fin):
        regr1 = RandomForestRegressor(max_depth=i, random_state=0)
        regr1.fit(Xtr, ytr)
        predictT = regr1.predict(Xvtaux)
        valor = evalRandomForest(yvtaux, predictT)
        if valor < best:
            best = valor
            posbest = i
    return(posbest, best)
        

In [ ]:
def train_randomForest(inih, finh, inid, find):
    posbest = 0
    best = 100
    for i in range(inih, finh):
        Xtrain, ytrain = preparar_datosRandomForest(df_train, i)
        Xvtaux, yvtaux = preparar_datosRandomForest(df_valitest, i)
        valores = train_randomForestdepth(inid, find, Xtrain, ytrain, Xvtaux, yvtaux)
        valor = valores[1]
        print(str(i)+" "+str(valores[0])+" "+str(valor))
        with open('OptimizaciónRandomForestIMC.txt', 'a') as archivo:
            archivo.write("Número de horas: "+str(i)+" Profundidad: "+str(valores[0])+" Valor de emp obtenido: "+str(valor) + "\n")
        if valor < best:
            best = valor
            posbest = valores[0]
    return(posbest, best)

In [16]:
tuple = train_randomForest(1, 21, 1, 150)

C:\Users\raulg\AppData\Local\Temp\ipykernel_5376\2914413355.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux1.append(aux[r])


KeyboardInterrupt: 

In [ ]:
print(tuple[0])
print(tuple[1])

9
0.7791437185511273


# Predicción Utilizando XGBoost

In [17]:
def create_df_n(df, n):
    df_aux = df.copy()
    for i in range(1, n):
        df_aux['open_before' + str(i)] = df_aux['open'].shift(+i)
        df_aux['high_before' + str(i)] = df_aux['high'].shift(+i)
        df_aux['low_before' + str(i)] = df_aux['low'].shift(+i)
        df_aux['close_before' + str(i)] = df_aux['close'].shift(+i)
        df_aux['value_before' + str(i)] = df_aux['value'].shift(+i)
    df_aux['close_next'] = df_aux['close'].shift(-1)
    df_aux = df_aux.dropna()
    return df_aux

Ponemos los datos en el formato correcto

In [18]:
df_xgb = create_df_n(df, 1)


In [19]:
tamanio_xgb = df_xgb.shape[0]
tamanio_xgb

36399

In [20]:
def createdftrain(df_aux):
    tamanio_aux = df_aux.shape[0]
    return df_aux.copy().loc[0:int(tamanio_aux*0.7)]

In [21]:
def createdfvali(df_aux):
    tamanio_aux = df_aux.shape[0]
    return df_aux.copy().loc[int(tamanio_aux*0.7 + 1):int(tamanio_aux*0.9)]

In [22]:
def createdftest(df_aux):
    tamanio_aux = df_aux.shape[0]
    return df_aux.copy().loc[int(tamanio_aux*0.9 + 1):tamanio_aux]

In [23]:
def preparar_datosXGBoost(df_aux):
    X = df_aux.drop(['date', 'close_next'], axis=1)
    #X = df_aux[['open', 'high', 'low', 'close', 'open_before1', 'high_before1', 'low_before1', 'close_before1']] 
    y = df_aux['close_next']
    return (xgb.DMatrix(data=X, label=y), y)

In [24]:
df_train_xgb = createdftrain(df_xgb)
df_train_xgb

,date,open,high,low,close,value,close_next
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,75,3.15
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,75,3.30
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,75,3.30
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,75,3.25
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,75,3.25
...,...,...,...,...,...,...,...
25475,2023-07-11 20:00:00,22.10,22.10,21.95,22.00,57,22.00
25476,2023-07-11 21:00:00,22.00,22.05,21.90,22.00,57,22.00
25477,2023-07-11 22:00:00,22.00,22.10,21.90,22.00,57,21.95
25478,2023-07-11 23:00:00,22.00,22.05,21.75,21.95,57,22.05


In [25]:
df_vali_xgb = createdfvali(df_xgb)
df_vali_xgb

,date,open,high,low,close,value,close_next
25480,2023-07-12 01:00:00,22.05,22.15,22.00,22.10,64,22.15
25481,2023-07-12 02:00:00,22.10,22.30,22.05,22.15,64,22.10
25482,2023-07-12 03:00:00,22.15,22.25,22.10,22.10,64,22.00
25483,2023-07-12 04:00:00,22.10,22.10,22.00,22.00,64,21.95
25484,2023-07-12 05:00:00,22.00,22.00,21.90,21.95,64,22.00
...,...,...,...,...,...,...,...
32755,2024-05-10 04:00:00,153.40,153.80,152.70,153.65,66,153.95
32756,2024-05-10 05:00:00,153.65,154.35,152.85,153.95,66,153.75
32757,2024-05-10 06:00:00,153.95,154.70,153.45,153.75,66,153.30
32758,2024-05-10 07:00:00,153.75,154.10,152.30,153.30,66,154.95


In [26]:
df_test_xgb = createdftest(df_xgb)
df_test_xgb

,date,open,high,low,close,value,close_next
32760,2024-05-10 09:00:00,154.95,155.75,154.25,154.35,66,154.10
32761,2024-05-10 10:00:00,154.35,154.50,153.45,154.10,66,154.15
32762,2024-05-10 11:00:00,154.10,154.80,153.25,154.15,66,154.15
32763,2024-05-10 12:00:00,154.15,154.30,153.25,154.15,66,155.05
32764,2024-05-10 13:00:00,154.15,155.20,153.00,155.05,66,153.30
...,...,...,...,...,...,...,...
36394,2024-10-08 19:00:00,143.80,143.95,142.40,143.35,49,142.95
36395,2024-10-08 20:00:00,143.35,143.90,142.35,142.95,49,143.75
36396,2024-10-08 21:00:00,142.95,144.10,142.25,143.75,49,144.50
36397,2024-10-08 22:00:00,143.75,144.50,143.35,144.50,49,144.25


In [27]:
df_valitest_xgb = pd.concat([df_vali_xgb, df_test_xgb], ignore_index=True)

In [28]:
XvtXGB = df_valitest_xgb[['open', 'high', 'low', 'close', 'value']] 
yvtXGB = df_valitest_xgb['close_next']

In [29]:
dtrainvali = xgb.DMatrix(data=XvtXGB, label=yvtXGB)

Definimos las características para el entrenamiento

Comprobación de la tasa de error en los datos de test

In [50]:
def evalXGB(Test_xgb, predict_xgb_test):
    suma = 0
    n = len(Test_xgb)
    for i in range(0,n):
        suma = abs(predict_xgb_test[i] - Test_xgb[i])/Test_xgb[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

In [51]:
def train_XGB_depth(maxdepthmin, maxdepthmax, dtrainf, dvalif, dtestf, ytest):
    best = 100
    best_depth = 0
    for i in range(maxdepthmin, maxdepthmax):
        param = {'max_depth': i, 'eta': 0.1, 'objective': 'reg:squarederror'}
        evals = [(dtrainf, 'train'), (dvalif, 'validacion')]
        bstaux = xgb.train(param, dtrainf, num_boost_round=1000, evals=evals, early_stopping_rounds=10, verbose_eval=10)
        predict_xgb_test = bstaux.predict(dtestf)
        valor = evalXGB(ytest, predict_xgb_test)
        if(valor < best):
            best = valor
            best_depth = i
    return (best_depth, best)

In [52]:
def trainGlobalXGB(inid, find, inih, finh):
    best = 100
    best_depth = 0
    for i in range(inih, finh):
        df_aux = create_df_n(df, i)
        dtrain_aux = createdftrain(df_aux)
        dvali_aux = createdfvali(df_aux)
        dtest_aux = createdftest(df_aux)
        dtrain_prep = preparar_datosXGBoost(dtrain_aux)
        dvali_prep = preparar_datosXGBoost(dvali_aux)
        dtest_prep = preparar_datosXGBoost(dtest_aux)
        values = train_XGB_depth(inid, find, dtrain_prep[0], dvali_prep[0], dtest_prep[0], dtest_prep[1].values)
        print(str(i)+" "+str(values[0])+" "+str(values))
        with open('OptimizaciónXGBoostIMC.txt', 'a') as archivo:
            archivo.write("Número de horas: "+str(i)+" Profundidad: "+str(values[0])+" Valor de emp obtenido: "+str(values[1]) + "\n")
        if(values[1] < best):
            best = values[1]
            best_depth = values[0]
    return best, best_depth

In [53]:
data = trainGlobalXGB(1, 150, 1, 21)

[0]	train-rmse:53.26953	validacion-rmse:55.23855
[10]	train-rmse:26.63411	validacion-rmse:21.66486
[20]	train-rmse:15.80944	validacion-rmse:9.74252
[30]	train-rmse:10.10769	validacion-rmse:6.02822
[40]	train-rmse:6.72877	validacion-rmse:4.32861
[50]	train-rmse:4.62899	validacion-rmse:3.40639
[60]	train-rmse:3.34697	validacion-rmse:2.84340
[70]	train-rmse:2.60039	validacion-rmse:2.59542
[80]	train-rmse:2.18529	validacion-rmse:2.50218
[90]	train-rmse:1.96376	validacion-rmse:2.44816
[100]	train-rmse:1.84537	validacion-rmse:2.42617
[110]	train-rmse:1.78430	validacion-rmse:2.41209
[120]	train-rmse:1.75328	validacion-rmse:2.41156
[130]	train-rmse:1.73739	validacion-rmse:2.40931
[140]	train-rmse:1.72916	validacion-rmse:2.40837
[150]	train-rmse:1.72463	validacion-rmse:2.40787
[160]	train-rmse:1.72172	validacion-rmse:2.40240
[170]	train-rmse:1.71903	validacion-rmse:2.39560
[180]	train-rmse:1.71644	validacion-rmse:2.38770
[190]	train-rmse:1.71391	validacion-rmse:2.37937
[200]	train-rmse:1.71147	

KeyboardInterrupt: 